# **Tarea 3**

En este taller se trabajará sobre la modificación del juego de escaleras y serpientes del taller anterior. Utilizando el módulo desarrollado en el taller anterior.

Se definen las librerias requeridas

In [7]:
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
import pandas as pd
from IPython.display import display

## _Punto 2:_

### Escriba un método que implemente el algoritmo Q-Learning. La política de comportamiento debe ser un parámetro de entrada al método.